In [ ]:
!pip install crewai crewai-tools langchain-groq

lib/python3.11/dist-packages (from chromadb>=0.5.23->crewai) (1.35.0)


In [ ]:
!pip install sentence-transformers transformers --upgrade

ERROR: Operation cancelled by user


In [ ]:
pip show crewai

In [ ]:
!pip uninstall langchain langchain-community
!pip install langchain-google-genai==1.0.10


Found existing installation: langchain 0.3.26
Uninstalling langchain-0.3.26:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/langchain-0.3.26.dist-info/*
    /usr/local/lib/python3.11/dist-packages/langchain/*
Proceed (Y/n)? Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/uninstall.py", line 106, in run
    uninstall_pathset = req.uninstall(
                        ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 722, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_uninstall.py", line 364, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
import os
import logging
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

# Set up logging for debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1️⃣ Setup Gemini LLM
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise EnvironmentError("GOOGLE_API_KEY environment variable not set")

llm = ChatGoogleGenerativeAI(
    google_api_key=google_api_key,
    model="gemini/gemini-1.5-flash",  # Use Gemini 1.5 Flash with provider prefix
    temperature=0.7  # Avoid temperature=0 to prevent potential agent looping
)

# 2️⃣ Input code
input_code = """
def fibonacci_iterative(n):
    if n < 0:
        return 0
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    fib_sequence = [0, 1]
    for i in range(2, n):
        next_fib = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_fib)
    return fib_sequence
"""

# 3️⃣ Agents
code_analyzer = Agent(
    role="Code Analyzer",
    goal="Analyze Python code to identify syntax or logic errors.",
    backstory="An expert Python code reviewer with a keen eye for bugs and edge cases.",
    llm=llm,
    verbose=True,
    memory=False  # Disable memory to avoid OpenAI embedding dependency
)

code_corrector = Agent(
    role="Code Corrector",
    goal="Fix bugs and improve code based on analysis.",
    backstory="A skilled Python developer specializing in code optimization and bug fixing.",
    llm=llm,
    verbose=True,
    memory=False  # Disable memory to avoid OpenAI embedding dependency
)

# 4️⃣ Tasks
analysis_task = Task(
    description=f"""
    Analyze the following Python code and identify any syntax or logic errors. Provide a detailed list of issues found:
    {input_code}
    """,
    agent=code_analyzer,
    expected_output="A detailed list of syntax or logic errors in the provided code."
)

correction_task = Task(
    description=f"""
    Based on the analysis from the previous task, fix the bugs in the provided code and output only the corrected version.
    Original code:
    {input_code}
    """,
    agent=code_corrector,
    expected_output="The corrected Python code as a string.",
    context=[analysis_task]  # Ensure correction_task uses analysis_task output
)

# 5️⃣ Create and execute crew
try:
    crew = Crew(
        agents=[code_analyzer, code_corrector],
        tasks=[analysis_task, correction_task],
        verbose=True,  # Boolean for detailed logging
        process="sequential",  # Explicitly set sequential process
        memory=False  # Disable crew-level memory to avoid OpenAI dependency
    )

    # 6️⃣ Run the crew
    logging.info("Starting crew execution")
    result = crew.kickoff()
    logging.info("Crew execution completed")

    # 7️⃣ Print results
    print("\n=== STEP 1: Analyzer Output ===")
    print(analysis_task.output.raw)
    print("\n=== STEP 2: Corrector Output ===")
    print(correction_task.output.raw)

except Exception as e:
    logging.error(f"Error during crew execution: {str(e)}")
    print(f"Error: {str(e)}")
    print("Please check your GOOGLE_API_KEY, network connection, and dependency versions.")